In [1]:
from pathlib import Path
import random
from functools import reduce

In [2]:
class Cell:

    # определяет ячейку, у каждой ячейки есть свое значение и местоположение
    def __init__(self, row: int, col: int, box: int) -> None:
        self.row = row
        self.col = col
        self.box = box

        self.value = 0

    # возвращает строковое представление ячейки
    def __str__(self) -> None:
        temp = (self.value, self.row, self.col, self.box)
        return "Value: %d, Row: %d, Col: %d, Box: %d" % temp

In [3]:
class Board:

    # инициализирует поле
    def __init__(self, numbers=None) -> None:
        self.rows = {}
        self.columns = {}
        self.boxes = {}
        self.cells = []

        # цикл для строк
        for row in range(0, 9):
            # цикл для колонок
            for col in range(0, 9):
                # вычисление блока
                box = 3 * (row // 3) + (col // 3)

                # создание экземпляра ячейки
                cell = Cell(row, col, box)

                # установление значения ячейки
                if numbers is not None:
                    cell.value = numbers.pop(0)
                else:
                    cell.value = 0
                if row not in self.rows:
                    self.rows[row] = []
                if col not in self.columns:
                    self.columns[col] = []
                if box not in self.boxes:
                    self.boxes[box] = []

                self.rows[row].append(cell)
                self.columns[col].append(cell)
                self.boxes[box].append(cell)
                self.cells.append(cell)

    # возвращает ячейки, не равные 0
    def get_used_cells(self) -> list:
        return [x for x in self.cells if x.value != 0]

    # возвращает ячейки, равные 0
    def get_unused_cells(self) -> list:
        return [x for x in self.cells if x.value == 0]

    # возвращает все возможные значения, которые могли бы быть присвоены
    # ячейка указанна в качестве аргумента
    def get_possibles(self, cell) -> list:
        possibilities = self.rows[cell.row] + self.columns[cell.col] + self.boxes[cell.box]
        excluded = set([x.value for x in possibilities if x.value != 0 and x.value != cell.value])
        return [x for x in range(1, 10) if x not in excluded]

    # вычисляет плотность значения ячейки
    def get_density(self, cell: Cell) -> float:
        possibilities = self.rows[cell.row] + self.columns[cell.col] + self.boxes[cell.box]
        if cell.value != 0:
            possibilities.remove(cell)
        return len([x for x in set(possibilities) if x.value != 0]) / 20.0

    # получает набор возможных значений, которые не могут быть в ячейке
    def get_excluded(self, cell: Cell) -> set:
        possibilities = self.rows[cell.row] + self.columns[cell.col] + self.boxes[cell.box]
        return set([x.value for x in possibilities if x.value != 0 and x.value != cell.value])

    # меняет местами две строки
    def swap_row(self, row_index1: int, row_index2: int, allow=False) -> None:
        if allow or row_index1 // 3 == row_index2 // 3:
            for x in range(0, len(self.rows[row_index2])):
                temp = self.rows[row_index1][x].value
                self.rows[row_index1][x].value = self.rows[row_index2][x].value
                self.rows[row_index2][x].value = temp
        else:
            raise Exception('Tried to swap non-familial rows.')

    # меняет местами две колонки
    def swap_column(self, col_index1: int, col_index2: int, allow=False) -> None:
        if allow or col_index1 // 3 == col_index2 // 3:
            for x in range(0, len(self.columns[col_index2])):
                temp = self.columns[col_index1][x].value
                self.columns[col_index1][x].value = self.columns[col_index2][x].value
                self.columns[col_index2][x].value = temp
        else:
            raise Exception('Tried to swap non-familial columns.')

    # меняет местами два стека
    def swap_stack(self, stack_index1: int, stack_index2: int) -> None:
        for x in range(0, 3):
            self.swap_column(stack_index1 * 3 + x, stack_index2 * 3 + x, True)

    # меняет местами две полосы
    def swap_band(self, band_index1: int, band_index2: int) -> None:
        for x in range(0, 3):
            self.swap_row(band_index1 * 3 + x, band_index2 * 3 + x, True)

    # создает копию доски
    def copy(self) -> 'Board':
        b = Board()
        for row in range(0, len(self.rows)):
            for col in range(0, len(self.columns)):
                b.rows[row][col].value = self.rows[row][col].value
        return b

    # возвращает строковое представление
    def __str__(self) -> str:
        output = []
        for index, row in self.rows.items():
            my_set = map(str, [x.value for x in row])
            new_set = []
            for x in my_set:
                if x == '0':
                    new_set.append("_")
                else:
                    new_set.append(x)
            output.append('|'.join(new_set))
        return '\r\n'.join(output)

    def save_to_file(self, path_to_file: Path) -> None:
        with open(path_to_file, 'w') as f:
            f.write(self.__str__())

In [4]:
class Solver:

    # конструктор, сохраняющий локальную копию доски
    def __init__(self, board: Board) -> None:
        self.board = board.copy()
        self.vacants = self.board.get_unused_cells()

    # проверяет содерржание каждого отсека
    def is_valid(self) -> bool:
        valid = set(range(1, 10))
        for i, box in self.board.boxes.items():
            if not valid == set([x.value for x in box]):
                return False
        for i, row in self.board.rows.items():
            if not valid == set([x.value for x in row]):
                return False
        for i, col in self.board.columns.items():
            if not valid == set([x.value for x in col]):
                return False
        return True

    # решает судоку, перемещаясь вперед назад, тестовые значения
    def solve(self) -> bool:
        index = 0
        while -1 < index < len(self.vacants):
            current = self.vacants[index]
            flag = False
            my_range = range(current.value + 1, 10)
            for x in my_range:
                if x in self.board.get_possibles(current):
                    current.value = x
                    flag = True
                    break
            if not flag:
                current.value = 0
                index -= 1
            else:
                index += 1
        if len(self.vacants) == 0:
            return False
        else:
            return index == len(self.vacants)

In [5]:
class Generator:

    # конструктор для генератора
    def __init__(self, starting_file: Path) -> None:
        f = open(starting_file)
        numbers = filter(lambda x: x in '123456789', list(reduce(lambda x, y: x + y, f.readlines())))
        numbers = list(map(int, numbers))
        f.close()
        self.board = Board(numbers)

    # рандомизирует готовое судоку
    def randomize(self, iterations: int) -> None:
        if len(self.board.get_used_cells()) == 81:

            # цикл по итерациям
            for x in range(0, iterations):

                # берет рандомную колонку или строку
                case = random.randint(0, 4)

                # берет рандомную полосу или стек
                block = random.randint(0, 2) * 3

                # чтобы выбрать строку и столбец мы перетасовываем массив и берем оба элемента
                options = list(range(0, 3))
                random.shuffle(options)
                piece1, piece2 = options[0], options[1]

                # выполнение преобразования доски
                if case == 0:
                    self.board.swap_row(block + piece1, block + piece2)
                elif case == 1:
                    self.board.swap_column(block + piece1, block + piece2)
                elif case == 2:
                    self.board.swap_stack(piece1, piece2)
                elif case == 3:
                    self.board.swap_band(piece1, piece2)
        else:
            raise Exception('Rearranging partial board may compromise uniqueness.')

    def reduce_via_logical(self, cutoff=81) -> None:
        cells = self.board.get_used_cells()
        random.shuffle(cells)
        for cell in cells:
            if len(self.board.get_possibles(cell)) == 1:
                cell.value = 0
                cutoff -= 1
            if cutoff == 0:
                break

    # проверка уникального решения (удаление ячейки)
    def reduce_via_random(self, cutoff=81) -> None:
        temp = self.board
        existing = temp.get_used_cells()

        # сортировка ячеек по плотности
        new_set = [(x, self.board.get_density(x)) for x in existing]
        elements = [x[0] for x in sorted(new_set, key=lambda x: x[1], reverse=True)]

        # для каждой ячейки в отсортированном списке
        for cell in elements:
            original = cell.value

            # получение списка значений, которые можно попробовать на месте
            complement = [x for x in range(1, 10) if x != original]
            ambiguous = False

            # проверка каждого значения чтобы попробовать его подставить
            for x in complement:

                # значение равно текущему х
                cell.value = x

                # создание экземпляра класса Solver
                s = Solver(temp)

                # если Solver может заполнить каждое поле и решение является допустимым, то
                # судоку становится неоднозначной после удаления определенной ячейки, так что мы можем выйти из нее
                if s.solve() and s.is_valid():
                    cell.value = original
                    ambiguous = True
                    break

            # если все значения были проверены и головоломка остается уникальной, мы можем удалить ее
            if not ambiguous:
                cell.value = 0
                cutoff -= 1

            # если мы достигаем лимита, выходим
            if cutoff == 0:
                break

    # возвращает текущее состояние генератора, включая количество пустых ячеек и представление части судоку
    def get_current_state(self) -> str:
        template = "There are currently %d starting cells.\n\rCurrent puzzle state:\n\r\n\r%s\n\r"
        return template % (len(self.board.get_used_cells()), self.board.__str__())

In [6]:
difficulties = {
    'easy': (35, 0),
    'medium': (81, 5),
    'hard': (81, 10),
    'extreme': (81, 15)
}

# получение желаемой сложности
difficulty = difficulties['easy']

# построение объекта генератора
gen = Generator(Path('test.txt'))

# применение 100 случайных преобразований к судоку
gen.randomize(100)

# получение копии до удаления слотов
initial = gen.board.copy()

# применение логического сокращения с соответствующим ограничением сложности
gen.reduce_via_logical(difficulty[0])

# нулевой случай
if difficulty[1] != 0:
    # применение случайного сокращения с соответствующим ограничением сложности
    gen.reduce_via_random(difficulty[1])


# получение копии после завершения сокращений
final = gen.board.copy()

# распечатка доски
print("{0}".format(final))
gen.board.save_to_file('result.txt')

9|_|_|3|6|5|_|2|_
6|_|_|_|_|4|_|8|7
_|2|4|_|9|_|6|3|_
4|_|_|_|_|_|7|9|8
_|9|8|_|4|2|_|_|1
3|5|1|9|_|8|_|6|2
2|1|_|7|8|_|5|4|6
_|_|6|1|_|3|8|_|9
_|7|9|_|5|_|2|_|_
